# Simple 2D cantilever beam example modelled with gmsh
In a first step, we have to import the reuired python modules
- HierAMuS: The finite element code
- gmsh: The mesher

In [ ]:
import HierAMuS
import gmsh

## Creating the model with gmsh
First initialize gmsh, then add a model to gmsh

In [ ]:
gmsh.initialize()

gmsh.model.add('CantileverBeam')

Next, define the geometry, which can be parametrized. This will be done by first adding points to gmsh. Here, the occ geometry kernel of gmsh will be used.

In [ ]:
L=10
h=1
gmsh.model.occ.addPoint(0,0,0)
gmsh.model.occ.addPoint(L,0,0)
gmsh.model.occ.addPoint(L,h,0)
gmsh.model.occ.addPoint(0,h,0)

The next step is to define the boundary lines with gmsh.

In [ ]:
gmsh.model.occ.addLine(1,2)
gmsh.model.occ.addLine(2,3)
gmsh.model.occ.addLine(3,4)
gmsh.model.occ.addLine(4,1)


The input can be graphically checked by first synchronizing the geometric model and the executing the graphical user interface of gmsh

In [ ]:
gmsh.model.occ.synchronize()
#gmsh.fltk.run()

Next, the definition of the faces is required. This is done in gmsh by first define a curveloop and afterwards by adding the face

In [ ]:
gmsh.model.occ.addCurveLoop([1,2,3,4])
gmsh.model.occ.addPlaneSurface([1])

Before creating the mesh, the geometry needs to be synchronized again

In [ ]:
gmsh.model.occ.synchronize()
#gmsh.fltk.run()

Next, the settings for quadrilateral element meshing needs to be done. Here, we use a transfinite mesh, by settings the necessary property on the lines and the face

In [ ]:
nx = 10 # number of nodes in x-direction
ny = 4  # number of nodes in y-direction
gmsh.model.mesh.setTransfiniteCurve(1,nx)
gmsh.model.mesh.setTransfiniteCurve(3,nx)
gmsh.model.mesh.setTransfiniteCurve(2,ny)
gmsh.model.mesh.setTransfiniteCurve(4,ny)
gmsh.model.mesh.setTransfiniteSurface(1)

gmsh.option.setNumber("Mesh.RecombineAll", 1) # This option sets gmsh to recombine triangles to quadrangles

gmsh.model.mesh.generate(2)

#gmsh.fltk.run() # check the mesh

# Initialize the finite element program
First create the finite element object, then set the solution type and afterwards the solver.

In [ ]:
fesys = HierAMuS.FEMPy("./","test")   # Create the FEMPy-object, including the specification of a folder and a filename to which data is written.
fesys.setStaticSolutionState() # Sets the solution state to static solution
fesys.setSolver(2) # Sets the solver to Eigen LDLT solver

Next, we can extract the necessary commands from the finite element program

In [ ]:
mesh = fesys.getMeshCommands()    # General mesh commands
gm = mesh.getFromGMESH()          # Special gmsh commands
geo = mesh.getGeometryCommands()  # The geometry commands
macro = fesys.getMacroCommands()  # The macro commands

With the gmsh-commands gm, the gmsh mesh can be transfered to the finite element program. Gmsh does not create all necessary data required for the finite element program. These additional data will be created by checkGeometry command from the geometry commands geo.

In [ ]:
gm.addGeomFromGmsh(gmsh)
geo.checkGeometry()

Add elements and assign material formulation

In [ ]:
#gm.addFaceElements(gmsh,faceTags=1,material=1)
gm.addQuadrilateralFiniteElements(gmsh,order=1,faceTags=1,material=1)
#[et, enum, nt] = gmsh.model.mesh.getElements(2,1)


Adding element and materialformualtion and group it in a material

In [ ]:
mesh.getElementFormulations().addEL201_2DShell(num=1, meshiddisp=1, disporder=1, mode=1)
mesh.getMaterialFormulations().addMA3_2D_LinearElastic_Isotrop(number=1, E=100, nu=0.3, thickness=1, plainstrain=1)
mesh.addMaterial(1,1,1)

After necessary data are created, the degrees of freedom can be distributed

In [ ]:
mesh.setDegreesOfFreedom()

After the degrees of freedom are distributed, boundary conditions can be set.
Here, we clamp the left side of the beam, by
- Getting the edges of that side
- Settings the boundary conditions

The parameters for singleBC are:
- eltype: The geometric element type on which the boundary conditions should be set
- number: The geometric object numbers to set the boundary conditions on
- meshId: meshId of the degrees of freedom
- dofs: a list with 3 entries. If 0: no boundary condition will be set, if 1 a boundary condition will be set
- shapeOrder: Polynomial order of the shape functions approximating the solution field with the given meshId
- shapeType: The shape function type on which the boundary conditions should be set
- set: If True the boundary conditions will be overridden, otherwise only additional boundary condition will be considered

In [ ]:
from HierAMuS.HierAMuSPyWrapper import HierAMuSPyFEM
bounedges = gm.getEdgeNumbers(gmsh, 4)
mesh.getBoundaryConditions().singleBC(eltype=geo.edgeType(), number=bounedges, meshId=1, dofs=[1,1,1], shapeOrder=1,shapeType=HierAMuSPyFEM.Geometry.ShapeFunctionTypes.H1,set=True)

In [ ]:
loadedges = gm.getEdgeNumbers(gmsh, 2)
mesh.getBoundaryConditions().singleLoad(eltype=geo.edgeType(), number=loadedges, meshId=1, load=[0,1,0], propnum=1,add=True, localLoad=False, shapeorder=1,shapeType=HierAMuSPyFEM.Geometry.ShapeFunctionTypes.H1)

After the boundary conditions are set, the equation system and sparse matrix can be set up

In [ ]:
macro.sparseSetUp()

In [ ]:
macro.setPropFunction(1)
macro.setDt(1)
macro.timeincr()

In [ ]:
macro.newton()

In [ ]:
fesys.getPlotCommands().toFile()